# Start Store Cameras

In this tutorial we will explore using the mobilenet pipeline we created in step 3 to run inference on the frames in a Video and then draw the identified object's bounding boxes, classification and classification confidence.

In [1]:
import torch
import pickle
import wallaroo

import os
import numpy as np
import json
import requests
import time
import imutils
import cv2
from cv_store.cv_demo_utils import CVDemo
from cv_store.store import Store

ModuleNotFoundError: No module named 'cv_store.cv_demo_utils'

In [ ]:
wl = wallaroo.Client()

In [ ]:
ws = wl.list_workspaces()
for w in ws:
    if w.name() == 'computer-vision':
        wl.set_current_workspace(w)

In [ ]:
model_name = 'mobilenet'
onnx_model_path = 'models/mobilenet.pt.onnx'
mobilenet_model = wl.upload_model(model_name, onnx_model_path)

In [ ]:
deployment_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(2).memory("12Gi").build()

In [ ]:
pipeline_name = 'mobilenet-pp'
pipeline = wl.build_pipeline(pipeline_name) \
            .add_model_step(mobilenet_model) \
            .deploy(deployment_config = deployment_config)


In [ ]:
time.sleep(5) # needed to allow the pipeline to settle in.
pipeline_url = pipeline._deployment._url()
print(pipeline_url)


## Test the pipeline in a video stream

### Prepare input image

Next we will load a sample image and resize it to the width and height required for the object detector.

We will convert the image to a numpy ndim array and add it do a dictionary

In [ ]:

# The size the image will be resized to
width = 640
height = 480

# Only objects that have a confidence > confidence_target will be displayed on the image
cvDemo = CVDemo()

imagePath = 'images/input/example/example_01.jpg'

# The image width and height needs to be set to what the model was trained for.  In this case 640x480.
tensor, resizedImage = cvDemo.loadImageAndResize(imagePath, width, height)

# get npArray from the tensorFloat
npArray = tensor.cpu().numpy()

#creates a dictionary with the wallaroo "tensor" key and the numpy ndim array representing image as the value.
dictData = {"tensor": npArray.tolist()}


# Run inference using the SDK 

Now lets have the model detect the objects on the image by running inference and extracting the results 

In [ ]:
time.sleep(2)
infResults = pipeline.infer(dictData)
results = infResults[0].raw
results['original_data'] = None  # We are removing the input image json.  Not needed
results

In [ ]:
#pipeline.logs()


In [ ]:

startTime = time.time()
infResults = pipeline.infer(dictData)
endTime = time.time()

results = infResults[0].raw
results['original_data'] = None  # We are removing the input image json.  Not needed


### Detect and Classify Objects in Video using Wallaroo Shadow Deployment

Next we will load each frame in the input-video feedubg ut to the pipeline for inferencing.  Then using the results we will draw a bounding box around each identified object, print its classification, and the model's confidence that the prediction is accurate on the frame and save the frame to an output video.

As we are executing this notice the time it takes to process each frame.  In the next section we will discuss ways to improve this performance.

In [ ]:
cvDemo = CVDemo()

store = Store("Just Walk Out")

camera = {
    'name' : 'grocery',
    'src-loc' : 'videos/amazon-fresh-go.mp4',
    'dest-loc' : 'videos/grocery-amazon-fresh-go-inferenced.mp4',
    'fps' : 15,
    'width' :  640,
    'height' : 480,
    'endpoint-url' : pipeline_url,
    'inference' : 'WALLAROO_SDK', # "ONNX" or "WALLAROO_API" or "WALLAROO_SDK"
    'pipeline_name' : pipeline_name,
    'pipeline' : pipeline,
    'model_name' : model_name,
    'confidence-target' : 0.75,
    'color': CVDemo.AMBER
}
store.addCamera(camera)

stream = cv2.VideoCapture(camera['src-loc'])
#for cnt in range(0,200):
#    (grabbed, frame) = stream.read()
    
store.cameraList[0].runInferenceOnFrame(frame,camera)

enabled = True
#store.setObjectDetection(enabled)
#store.setAnomolyDetection(enabled)
#store.setEnableDriftDetection(enabled)
#store.setTrack(['person','hand bag'])
#store.startCameras()

    

### Conclusion

Notice how simple it is to take the mobilenet object detectors rest api endpoint url and use it in a video stream.  Now its your turn.  Upload a video to this notebook and replace the input-video with the path of the uploaded video.  Update the output video accordingly.

See how well it works for you.  Good luck!


In [ ]:
#pipeline.undeploy()
for d in wl.list_deployments():
    d.undeploy()
    